In [ ]:
import requests
from pyambit.datamodel import Substances, Study 
import nexusformat.nexus.tree as nx
import os.path
import tempfile
# to_nexus is not added without this import
from pyambit import nexus_writer
import json

In [ ]:
def query(url = "https://apps.ideaconsult.net/gracious/substance/" ,params = {"max" : 1}):
    substances = None
    headers = {'Accept': 'application/json'}
    result = requests.get(url,params=params,headers=headers)
    if result.status_code==200:
        response =  result.json()
        substances = Substances.model_construct(**response)
        for substance in substances.substance:
            url_study = "{}/study".format(substance.URI)
            study = requests.get(url_study,headers=headers)
            if study.status_code==200:
                response_study = study.json()
                substance.study = Study.model_construct(**response_study).study

    return substances

def write_studies_nexus(substances):
    for substance in substances.substance:
        for study in substance.study:
            file = os.path.join(tempfile.gettempdir(), "study_{}.nxs".format(study.uuid))
            nxroot = nx.NXroot()
            try:
                study.to_nexus(nxroot)
                nxroot.save(file, mode="w")
            except Exception as err:
                #print("error",file,str(err))
                print(file)

In [ ]:

try:
    substances = query(params = {"max" : 10})   
    _json = substances.model_dump(exclude_none=True)
    new_substances = Substances.model_construct(**_json)
    #test roundtrip
    assert substances == new_substances

    file = os.path.join(tempfile.gettempdir(), "remote.json")
    print(file)
    with open(file, 'w', encoding='utf-8') as file:
        file.write(substances.model_dump_json(exclude_none=True))
    write_studies_nexus(substances)
except Exception as x:
    print(x)